In [4]:
from IPython.display import HTML
from IPython.display import display, Image


<h1 align="center" style="font-size: 35px;"><font color="C1772D"><strong>Aplicación del Control Óptimo en el Campo de la Robótica</strong></font></h1>

<h2 align="center"><font color="#667388"><strong>Luis F. Recalde</strong><br>Ambato-Ecuador</font></h2>


<h3 align="center"><font color="#667388"><strong>Universidad Indoamerica</strong></font></h3>

<h3 align="left" style="font-size: 30px;"><font color="C1772D"><strong>Sistemas Dinámicos</strong></font></h3>


<h3 align="left" style="font-size: 20px;"><font color="#667388"><strong>Sistemas Dinámicos en tiempo discreto.</strong></font></h3>


<div class="alert alert-block alert-warning" style="font-size: 16px;">  
  <b>Nota:</b> En términos generales, no es posible resolver de manera analítica ecuaciones diferenciales no lineales cuando se presentan en la siguiente forma: $\dot{\mathbf{x}} = \mathbf{f}(\mathbf{x})$
</div>

<div class="alert alert-block alert-warning" style="font-size: 16px;">  
  <b>Nota:</b> Si es posible resolver ecuaciones diferenciales lineales; sin embargo, los sistemas que nos interesan, como robots móviles y manipuladores, se representan mediante ecuaciones diferenciales no lineales.
</div>

<div class="alert alert-block alert-warning" style="font-size: 16px;">  
  <b>Nota:</b> Nuestro computador debe ser capaz de operar con estas dinámicas; por lo tanto, debemos utilizar técnicas de discretización.
</div>

<div class="alert alert-block alert-warning" style="font-size: 16px;">  
  <b>Nota:</b> Los modelos discretos son capaces de capturar ciertos efectos que no es posible representar en modelos continuos.
</div>

<div style="text-align: justify; font-size: 20px;">Existen diferentes formas de escribir las dinámicas discretas, las cuales pueden ser:</div>


<h3 align="left" style="font-size: px;"><font color="#2D5AC1"><strong>Discretización explícita:</strong></font></h3>


<div style="text-align: justify; font-size: 20px;">Su representación viene dada por la siguiente forma:</div>


<span style="font-size:1.4em;">
\begin{equation}
    {\mathbf{x}_{k+1}} = \mathbf{f}_d(\mathbf{x}_{k}, \mathbf{u}_{k})
\end{equation}
</span>

<div style="text-align: justify; font-size: 20px;">Donde $\mathbf{x}_k$ y $\mathbf{u}_k$ representan los estados y acciones de control en un instante, mientras que $\mathbf{x}_{k+1}$ representa los estados del sistema en un instante futuro.</div>


<div style="text-align: justify; font-size: 20px;">En la literatura existen diferentes formas de obtener una discretización explícita; una de las más simples es la "Forward Euler Integration". Este método puede ser representado de la siguiente forma:</div>


<span style="font-size:1.4em;">
\begin{equation}
    {\mathbf{x}_{k+1}} = \overbrace{{\mathbf{x}_{k}} + t_s \mathbf{f}(\mathbf{x}_{k}, \mathbf{u}_{k})}^{\mathbf{f}_d(\mathbf{x}_{k}, \mathbf{u}_{k})}
\end{equation}
</span>

<div style="text-align: justify; font-size: 20px;"> considerando que $t_s$ es el tiempo de muestreo.</div>


<div style="text-align: justify; font-size: 20px;">A continuación, se presenta un ejemplo de este método.</div>


<h3 align="left" style="font-size: px;"><font color="#2D5AC1"><strong>Pendulo Simple</strong></font></h3>


<span style="font-size:1.4em;">
\begin{equation}
    \dot{\mathbf{x}} = \begin{bmatrix} \dot\theta \\ \ddot{\theta} \end{bmatrix} = \begin{bmatrix} \dot\theta \\ -\frac{g}{l}\sin{(\theta)} + \frac{1}{m l^2} u\end{bmatrix} =\begin{bmatrix} \mathbf{v} \\ -\frac{g}{l}\sin{(\mathbf{q})} + \frac{1}{m l^2} \mathbf{u}\end{bmatrix} 
\end{equation}
</span>

<span style="font-size:1.4em;">
\begin{equation}
    \dot{\mathbf{x}} = \mathbf{f}(\mathbf{x}, \mathbf{u})
\end{equation}
</span>

<span style="font-size:1.4em;">
\begin{equation}
    \mathbf{f}(\mathbf{x},\mathbf{u})~= \begin{bmatrix} \mathbf{v} \\ -\frac{g}{l}\sin{(\mathbf{q})} + \frac{1}{m l^2} \mathbf{u}\end{bmatrix} 
\end{equation}
</span>

<div class="alert alert-block alert-success" style="font-size: 16px;">  
  <b>Code:</b>
</div>

### Importar Librerias

In [5]:
import numpy as np
from numpy import linalg as LA

### Funciones